In [30]:
# ! conda install -c conda-forge notebook

In [31]:
# ! pip install easydict

In [60]:
import config
import json
import pickle

In [61]:

with open('/home/arunava/VQA_Med/train_dataset_pickle/train_dataset_df.pkl', 'rb') as f:
    data = pickle.load(f) 
 

# print(type(data))

In [62]:
data

image_id                                              image  \
0      synpic41148  [[[tensor(-1.3815), tensor(-1.3815), tensor(-1...   
1      synpic43984  [[[tensor(-2.1008), tensor(-2.1008), tensor(-2...   
2      synpic38930  [[[tensor(-0.3027), tensor(-0.3883), tensor(-0...   
3      synpic52143  [[[tensor(-1.6898), tensor(-1.7240), tensor(-1...   
4      synpic20934  [[[tensor(2.0434), tensor(2.0434), tensor(2.04...   
...            ...                                                ...   
12787  synpic35685  [[[tensor(-1.5699), tensor(-1.5699), tensor(-1...   
12788  synpic35687  [[[tensor(-1.5699), tensor(-1.5699), tensor(-1...   
12789  synpic35701  [[[tensor(-1.5699), tensor(-1.5699), tensor(-1...   
12790  synpic48036  [[[tensor(-2.1179), tensor(-2.1179), tensor(-2...   
12791  synpic54897  [[[tensor(-2.0837), tensor(-2.1008), tensor(-2...   

                                                Question  \
0                            what kind of image is this?   
1                           is this a t1 weighted image?   
2      what type of imaging modality is used to acqui...   
3                             is this a noncontrast mri?   
4                   what type of image modality is this?   
...                                                  ...   
12787                              is this image normal?   
12788             is there something wrong in the image?   
12789            is there evidence of any abnormalities?   
12790                             is the ct scan normal?   
12791                is there an abnormality in the mri?   

                     Answer  labels  
0      cta - ct angiography      32  
1                        no       2  
2           us - ultrasound      15  
3                        no       2  
4           xr - plain film       5  
...                     ...     ...  
12787                   yes       3  
12788                    no       2  
12789                    no       2  
12790                   yes       3  
12791                    no       2  

[12792 rows x 5 columns]

In [68]:
c = data['labels'].unique().tolist()

In [69]:
len(c)

178

In [35]:
with open(config.path+'/VQA_Med/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

178


In [36]:
# ! pip install numpy

In [37]:
# ! pip install torch

In [38]:
# ! pip install nltk

In [39]:
# ! pip install pandas
# ! pip install matplotlib

In [40]:
# ! pip install torchvision

In [41]:
# ! pip install transformers

In [42]:
# ! pip install modin[ray] 

In [43]:
# ! pip install ray[tune]

In [44]:
# ! pip install ray[rllib]

In [45]:
import os
import argparse
import shutil
import numpy as np

import copy
import random
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.optim import lr_scheduler
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import bleu_score
from nltk.translate.bleu_score import corpus_bleu
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from data_loader import get_loader

from models import VqaClassifierModel


import matplotlib.pyplot as plt
import nltk
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
nltk.download('punkt')
# opt = config.parse_opt()

# print('opt',opt)

# seed_value = opt.SEED
# np.random.seed(seed_value)
# random.seed(seed_value)
# torch.manual_seed(seed_value)
# torch.cuda.manual_seed(seed_value)
# torch.cuda.manual_seed_all(seed_value)
# torch.backends.cudnn.enabled = False
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

seed_value = 97
np.random.seed(seed_value)
random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


[nltk_data] Downloading package punkt to /home/arunava/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    print('output.shape',output.shape)
    maxk = max(topk)
    print('maxk',maxk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    print('pred',pred.shape)
    # print('pred',pred)
    pred = pred.t()
    print('pred',pred.shape)



    
    if target.dim() == 2: # multians option
        _, target = torch.max(target, 1)

    print('target',target.view(1, -1).shape)
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    print('correct',correct.shape)
    
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        print('correct_k',correct_k)
        print(correct_k.shape)
        res.append((correct_k / batch_size))
   
    return res
        
 




In [47]:
def get_bleu_score(predicted, true_ans_text):
    path_output_change = config.path_output_chd
    with open(path_output_change+'/answer_classes.json', 'r') as j:
        answer_classes_dict = json.load(j)
    score = 0.0
    print('true_ans_text',true_ans_text)
    assert (len(predicted) == len(true_ans_text))
    ans_keys = list(answer_classes_dict.keys())
    ans_values = list(answer_classes_dict.values())
    

    for pred, true_ans in zip(predicted, true_ans_text):
        index_ans = ans_values.index(pred)
        
        score += sentence_bleu([true_ans.split(' ')], ans_keys[index_ans].split(' '), smoothing_function=bleu_score.SmoothingFunction().method2)

    return score/len(true_ans_text)


In [48]:
def load_checkpoint(model, optimizer, filename=None): 
    # Note: Input model & optimizer should be pre-defined. This routine only updates their states. 
    start_epoch = 0 
    if os.path.isfile(filename): 
        print("=> loading checkpoint '{}'".format(filename)) 
        checkpoint = torch.load(filename) 
        start_epoch = checkpoint['epoch'] 
        model.load_state_dict(checkpoint['state_dict']) 
        optimizer.load_state_dict(checkpoint['optimizer']) 
        print("=> loaded checkpoint '{}' (epoch {})" .format(filename,
                                                            checkpoint['epoch'])) 
    else: print("=> no checkpoint found at '{}'".format(filename)) 
    return model, optimizer, start_epoch
    

In [49]:
def make_plot(history, epoch_max, path_output_chd, type_plot='loss'):
    train = history['train']
    valid = history['valid']
    fig, ax = plt.subplots()
    epochs = range(epoch_max)
    
    
    if type_plot=='loss':
        plt.plot(epochs, train, '-r', lw=2, label='Training loss')
        plt.plot(epochs, valid, '-b',lw=2, label='validation loss')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.savefig(path_output_chd+'/imgs/loss.png')
        
    elif type_plot == 'acc1':
        ltrain = []
        lvalid = []
        for i in range(len(train)):
            ltrain.append(train[i].to('cpu').numpy().tolist())
        for i in range(len(valid)):
            lvalid.append(valid[i].to('cpu').numpy().tolist())

        plt.plot(epochs,ltrain, '-r', lw = 2, label='Training Top 1 Accuracy')
        plt.plot(epochs, lvalid, '-b', lw = 2, label='validation Top 1 Accuracy')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation Top 1 Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Top 1 Accuracy')
        plt.savefig(path_output_chd+'/imgs/acc1.png')

    elif type_plot == 'acc5':
    
        plt.plot(epochs, train, '-r', lw = 2, label='Training Top 5 Accuracy')
        plt.plot(epochs, valid, '-b', lw = 2, label='validation Top 5 Accuracy')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation Top 5 Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Top 5 Accuracy')
        plt.savefig(path_output_chd+'/imgs/acc5.png')
    else:
        plt.plot(epochs, train, '-r', lw = 2, label='Training blue')
        plt.plot(epochs, valid, '-b', lw = 2, label='validation blue')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation blue')
        plt.xlabel('Epochs')
        plt.ylabel('Blue')
        plt.savefig(path_output_chd+'/imgs/blue.png')

    
    
    plt.show()

In [50]:
import easydict

In [51]:
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1984,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":10,
    })

In [52]:
def mixup_data_vqa(v, q, a, alpha=1.0, use_cuda=True):
    '''Returns mixed images, pairs of questions, targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = v.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_v = lam * v + (1 - lam) * v[index, :]
    a_a, a_b = a, a[index]
    q_a, q_b = q, q[index]
    return mixed_v, a_a, a_b, q_a, q_b, lam


In [53]:

# import numpy as np
# import pandas as pd
import re
import json
# import torch
# import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import random
import torchvision
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel
import config
# device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

path_output_change =config.path_output_chd

with open(path_output_change+'/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


class BERTokenizer():

    def __init__(self,opt):
        
        # Load the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.opt = opt
    #pre-process the text data
    def text_preprocessing(self, text):

        # Remove trailing whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        return text


    # Create a function to tokenize a set of texts
    def preprocessing_for_bert(self, data):
        print('data',len(data))
        """Perform required preprocessing steps for pretrained BERT.
        @param    data (np.array): Array of texts to be processed.
        @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
        @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                    tokens should be attended to by the model.
        """
        # Create empty lists to store outputs
        input_ids = []
        attention_masks = []
        MAX_LEN = self.opt.MAX_QUESTION_LEN
        # For every sentence...
        for sent in data:
            
            encoded_sent = self.tokenizer.encode_plus(
                text=self.text_preprocessing(sent),  # Preprocess sentence
                add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
                max_length=MAX_LEN,                  # Max length to truncate/pad
                pad_to_max_length=True,         # Pad sentence to max length
                #return_tensors='pt',           # Return PyTorch tensor
                truncation=True,
                return_attention_mask=True      # Return attention mask
                )
            
            # Add the outputs to the lists
            input_ids.append(encoded_sent.get('input_ids'))
            attention_masks.append(encoded_sent.get('attention_mask'))

        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        attention_masks = torch.tensor(attention_masks)

        return input_ids, attention_masks


# Create the question encoder base on  BertClassfier
class BertQstEncoder(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, opt,freeze_bert=True):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        @param    opt: for configuration parameter
        """
        super(BertQstEncoder, self).__init__()
        self.opt = opt
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in= self.opt.BERT_UNIT_NUM

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert.eval()
        self.bert_emb = self.bert.embeddings
        self.bert_encode_layer1 = (self.bert.encoder.layer)[0]
        self.bert_encode_layer2 = (self.bert.encoder.layer)[1]
        self.bert_encode_layer3 = (self.bert.encoder.layer)[2]
        self.bert_encode_layer4 = (self.bert.encoder.layer)[3]
        self.bert_encode_layer5 = (self.bert.encoder.layer)[4]
        self.bert_encode_layer6 = (self.bert.encoder.layer)[5]
        self.bert_encode_layer7 = (self.bert.encoder.layer)[6]
        self.bert_encode_layer8 = (self.bert.encoder.layer)[7]
        self.bert_encode_layer9 = (self.bert.encoder.layer)[8]
        self.bert_encode_layer10 = (self.bert.encoder.layer)[9]
        self.bert_encode_layer11 = (self.bert.encoder.layer)[10]
        self.bert_encode_layer12 = (self.bert.encoder.layer)[11]

        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """


        # Feed input to BERT

        with torch.no_grad():
            emb_out = self.bert_emb(input_ids, attention_mask)
            layer1= self.bert_encode_layer1(emb_out)
            # print('layer1',layer1[0].shape)
            layer2= self.bert_encode_layer2(layer1[0])
            # print('layer2',layer2[0].shape)
            layer3= self.bert_encode_layer3(layer2[0])
            # print('layer3',layer3[0].shape)
            layer4= self.bert_encode_layer4(layer3[0])
            # print('layer4',layer4[0].shape)
            layer5= self.bert_encode_layer5(layer4[0])
            # print('layer5',layer5[0].shape)
            layer6= self.bert_encode_layer6(layer5[0])
            # print('layer6',layer6[0].shape)
            layer7= self.bert_encode_layer7(layer6[0])
            # print('layer7',layer7[0].shape)
            layer8= self.bert_encode_layer8(layer7[0])
            # print('layer8',layer8[0].shape)
            layer9= self.bert_encode_layer9(layer8[0])
            # print('layer9',layer9[0].shape)
            layer10= self.bert_encode_layer10(layer9[0])
            # print('layer10',layer10[0].shape)
            layer11= self.bert_encode_layer11(layer10[0])
            # print('layer11',layer11[0].shape)
            layer12= self.bert_encode_layer12(layer11[0])
            # print('layer12',layer12[0].shape)

        word_question_representation = (layer11[0] +layer12[0])/2
               
        return word_question_representation


#Extract the question feature with co-attention
class QuestionFeatureExtractionAtt(nn.Module):
    '''
        Extract the question with co-attention, get from https://github.com/asdf0982/vqa-mfb.pytorch 
    '''

    def __init__(self,opt):

        super(QuestionFeatureExtractionAtt, self).__init__()

        self.opt = opt
        self.NUM_QUESTION_GLIMPSE = self.opt.NUM_QUESTION_GLIMPSE

        self.JOINT_EMB_SIZE = self.opt.MFB_FACTOR_NUM * self.opt.MFB_OUT_DIM
        self.Softmax = nn.Softmax(dim=-1)

        self.Linear1_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM* self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Linear2_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM*self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        
        self.Dropout_M = nn.Dropout(p=self.opt.MFB_DROPOUT_RATIO)
        self.dropout = nn.Dropout(self.opt.BERT_DROPOUT_RATIO) 
        self.Conv1_Qatt = nn.Conv2d(self.opt.BERT_UNIT_NUM, self.opt.IMAGE_CHANNEL, 1) 
        self.Conv2_Qatt = nn.Conv2d(self.opt.IMAGE_CHANNEL, self.opt.NUM_QUESTION_GLIMPSE, 1)

    def forward(self,qst_encoding):

        # print('qst_encoding.shape',qst_encoding.shape)
        '''
        Question Attention
        '''   
        self.batch_size = qst_encoding.shape[0]

        qst_encoding = self.dropout(qst_encoding)
        # print('qst_encoding',qst_encoding.shape)
        qst_encoding_resh =  torch.unsqueeze(qst_encoding, 3)       # N=4 x 768 x T=14 x 1     
        # print('qst_encoding_resh',qst_encoding_resh.shape)
        qatt_conv1 = self.Conv1_Qatt(qst_encoding_resh)                   # N x 512 x T x 1
        # print('qst_conv1',qatt_conv1.shape)
        qatt_relu = F.relu(qatt_conv1)
        # print('qatt_relu',qatt_relu.shape)
        qatt_conv2 = self.Conv2_Qatt(qatt_relu)
        # print('qatt_conv2',qatt_conv2.shape)                          # N x 2 x T x 1
        qatt_conv2 = qatt_conv2.contiguous().view(self.batch_size*2,-1)
        # print('qatt_conv2',qatt_conv2.shape)
        qatt_softmax = self.Softmax(qatt_conv2)
        # print('qatt_softmax',qatt_softmax.shape)
        qatt_softmax = qatt_softmax.view(self.batch_size, 2, -1, 1)
        # print('qatt_softmax',qatt_softmax.shape)
        qatt_feature_list = []
        for i in range(self.NUM_QUESTION_GLIMPSE):
            t_qatt_mask = qatt_softmax.narrow(1, i, 1)              # N x 1 x T x 1
            t_qatt_mask = t_qatt_mask * qst_encoding_resh           # N x 768 x T x 1
            t_qatt_mask = torch.sum(t_qatt_mask, 2, keepdim=True)   # N x 768 x 1 x 1
            qatt_feature_list.append(t_qatt_mask)
        qatt_feature_concat = torch.cat(qatt_feature_list, 1)       # N x 1536 x 1 x 1

        return qatt_feature_concat


#Extract the image feature with MFB and co-attention 
class ImageFeatureExtractionAtt(nn.Module):

    '''
        Extract the image with co-attention, get from https://github.com/asdf0982/vqa-mfb.pytorch 
    '''

    def __init__(self,opt):
        super(ImageFeatureExtractionAtt, self).__init__()
        self.opt = opt
        self.MFB_FACTOR_NUM = self.opt.MFB_FACTOR_NUM
        self.MFB_OUT_DIM = self.opt.MFB_OUT_DIM
        self.NUM_IMG_GLIMPSE =self.opt.NUM_IMG_GLIMPSE
        self.IMG_FEAT_SIZE = self.opt.IMG_FEAT_SIZE
        
        self.JOINT_EMB_SIZE = self.opt.MFB_FACTOR_NUM * self.opt.MFB_OUT_DIM
        self.Softmax = nn.Softmax(dim=-1)

        self.Linear1_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM* self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Linear_i_proj = nn.Linear(self.opt.IMAGE_CHANNEL*self.opt.NUM_IMG_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Conv_i_proj = nn.Conv2d(self.opt.IMAGE_CHANNEL, self.JOINT_EMB_SIZE, 1)
        

        self.Dropout_M = nn.Dropout(p=self.opt.MFB_DROPOUT_RATIO)

        self.Conv1_Iatt = nn.Conv2d(self.opt.MFB_OUT_DIM, self.opt.IMAGE_CHANNEL, 1) # (1000, 512, 1)
        self.Conv2_Iatt = nn.Conv2d(self.opt.IMAGE_CHANNEL, self.NUM_IMG_GLIMPSE, 1)
    
    def forward(self, img_feature, qstatt_feature):

        '''
        Image Attention with MFB
        '''
        self.batch_size = img_feature.shape[0]
        q_feat_resh = torch.squeeze(qstatt_feature)                              # N x 1536
        i_feat_resh = img_feature.unsqueeze(3)                                   # N x 512 x 196 x 1
        iatt_q_proj = self.Linear1_q_proj(q_feat_resh)                                  # N x 5000
        iatt_q_resh = iatt_q_proj.view(self.batch_size, self.JOINT_EMB_SIZE, 1, 1)      # N x 5000 x 1 x 1
        iatt_i_conv = self.Conv_i_proj(i_feat_resh)                                     # N x 5000 x 196 x 1
        iatt_iq_eltwise = iatt_q_resh * iatt_i_conv
        iatt_iq_droped = self.Dropout_M(iatt_iq_eltwise)                                # N x 5000 x 196 x 1
        iatt_iq_permute1 = iatt_iq_droped.permute(0,2,1,3).contiguous()                 # N x 196 x 5000 x 1
        iatt_iq_resh = iatt_iq_permute1.view(self.batch_size, self.IMG_FEAT_SIZE, self.MFB_OUT_DIM, self.MFB_FACTOR_NUM)
        iatt_iq_sumpool = torch.sum(iatt_iq_resh, 3, keepdim=True)                      # N x 196 x 1000 x 1 
        iatt_iq_permute2 = iatt_iq_sumpool.permute(0,2,1,3)                             # N x 1000 x 196 x 1
        iatt_iq_sqrt = torch.sqrt(F.relu(iatt_iq_permute2)) - torch.sqrt(F.relu(-iatt_iq_permute2))
        iatt_iq_sqrt = iatt_iq_sqrt.reshape(self.batch_size, -1)                           # N x 196000
        iatt_iq_l2 = F.normalize(iatt_iq_sqrt)
        iatt_iq_l2 = iatt_iq_l2.view(self.batch_size, self.MFB_OUT_DIM, self.IMG_FEAT_SIZE, 1)  # N x 1000 x 196 x 1

        iatt_conv1 = self.Conv1_Iatt(iatt_iq_l2)                    # N x 512 x 196 x 1
        iatt_relu = F.relu(iatt_conv1)
        iatt_conv2 = self.Conv2_Iatt(iatt_relu)                     # N x 2 x 196 x 1
        iatt_conv2 = iatt_conv2.view(self.batch_size*self.NUM_IMG_GLIMPSE, -1)
        iatt_softmax = self.Softmax(iatt_conv2)
        iatt_softmax = iatt_softmax.view(self.batch_size, self.NUM_IMG_GLIMPSE, -1, 1)
        iatt_feature_list = []
        for i in range(self.NUM_IMG_GLIMPSE):
            t_iatt_mask = iatt_softmax.narrow(1, i, 1)              # N x 1 x 196 x 1
            t_iatt_mask = t_iatt_mask * i_feat_resh                 # N x 512 x 196 x 1
            t_iatt_mask = torch.sum(t_iatt_mask, 2, keepdim=True)   # N x 512 x 1 x 1
            iatt_feature_list.append(t_iatt_mask)
        iatt_feature_concat = torch.cat(iatt_feature_list, 1)       # N x 1024 x 1 x 1
        iatt_feature_concat = torch.squeeze(iatt_feature_concat)    # N x 1024
        return iatt_feature_concat



class VqaClassifierModel(nn.Module):
    '''
        Fusion with MFB,  get from https://github.com/asdf0982/vqa-mfb.pytorch 
    '''

    def __init__(self, opt):
        super(VqaClassifierModel, self).__init__()
        self.opt = opt
        
        self.JOINT_EMB_SIZE = self.opt.MFB_FACTOR_NUM * self.opt.MFB_OUT_DIM
        
        self.MFB_OUT_DIM = self.opt.MFB_OUT_DIM
        self.MFB_FACTOR_NUM = self.opt.MFB_FACTOR_NUM
        NUM_OUTPUT_UNITS = self.opt.NUM_OUTPUT_UNITS


        self.tokenizer = BERTokenizer(self.opt)
        self.bert_model = BertQstEncoder(self.opt)

       
        self.qst_feature_att = QuestionFeatureExtractionAtt(self.opt)
        self.img_feature_att = ImageFeatureExtractionAtt(self.opt)

        self.Linear2_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM*self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Linear_i_proj = nn.Linear(self.opt.IMAGE_CHANNEL*self.opt.NUM_IMG_GLIMPSE, self.JOINT_EMB_SIZE)

        self.Dropout_M = nn.Dropout(p=self.opt.MFB_DROPOUT_RATIO)

        self.Linear_predict = nn.Linear(self.opt.MFB_OUT_DIM, NUM_OUTPUT_UNITS)


    def forward(self, img, qst):

        self.batch_size = img.shape[0]
        image_feature = img
        print('image_feature.shape',image_feature.shape)
        input_ids, attention_mask = self.tokenizer.preprocessing_for_bert(qst)
        question_feature = self.bert_model(input_ids.to(device), attention_mask.to(device))
        print('question_feature.shape',question_feature.shape)
        question_feature = question_feature.transpose(1, 2)      # N=4 x 768 x T=14
        print('question_feature.shape',question_feature.shape)
        q_featatt = self.qst_feature_att(question_feature)      # N x 1536
        print('q_featatt',q_featatt.shape)
        iatt_feature_concat = self.img_feature_att(image_feature,q_featatt)        # N x 1024
        # print('iatt_feature_concat',iatt_feature_concat)
        
        '''
        Fine-grained Image-Question MFB fusion
        '''
        q_feat_resh = torch.squeeze(q_featatt)
        # print('q_feat_resh',q_feat_resh)
        mfb_q_proj = self.Linear2_q_proj(q_feat_resh)               # N x 5000
        # print('mfb_q_proj',mfb_q_proj)
        mfb_i_proj = self.Linear_i_proj(iatt_feature_concat)        # N x 5000
        # print('mfb_i_proj',mfb_i_proj)
        mfb_iq_eltwise = torch.mul(mfb_q_proj, mfb_i_proj)          # N x 5000
        # print('mfb_iq_eltwise',mfb_iq_eltwise)
        mfb_iq_drop = self.Dropout_M(mfb_iq_eltwise)
        # print('mfb_iq_drop',mfb_iq_drop)
        mfb_iq_resh = mfb_iq_drop.view(self.batch_size, 1, self.MFB_OUT_DIM, self.MFB_FACTOR_NUM)   # N x 1 x 1000 x 5
        mfb_iq_sumpool = torch.sum(mfb_iq_resh, 3, keepdim=True)    # N x 1 x 1000 x 1
        mfb_out = torch.squeeze(mfb_iq_sumpool)                     # N x 1000
        mfb_sign_sqrt = torch.sqrt(F.relu(mfb_out)) - torch.sqrt(F.relu(-mfb_out))
        mfb_l2 = F.normalize(mfb_sign_sqrt)
        prediction = self.Linear_predict(mfb_l2)
        prediction = F.log_softmax(prediction, -1) # N x num_class

        return prediction

In [54]:
class VqaDataset(data.Dataset):
    '''
        Main class use to retrieve our dataset from pickle file.
    '''

    def __init__(self, input_dir, input_vqa, img_feat_vqa, transform=None, phase = 'train'):
        self.input_dir = input_dir
        self.vqa = np.load(input_dir+'/'+input_vqa, allow_pickle=True )
        self.img_feat_vqa = np.load(input_dir+'/'+img_feat_vqa, allow_pickle=True )
        
        self.vocab_size = None
        self.phase = phase
        
   

    def __getitem__(self, idx):

        vqa = self.vqa
        img_feat_vqa = self.img_feat_vqa
        
        image_id = vqa['image_id'].values[idx]
        image_feat = torch.Tensor(img_feat_vqa[image_id])
        # print('image_feat',image_feat)
        # image = vqa['image'].values[idx]
        # print('image',type(image))
        qst2idc =  vqa['Question'].values[idx]
        sample = { 'image_feature':image_feat ,'question': qst2idc } 
        if (self.phase == 'train') or  (self.phase == 'valid'):
            ans2idc = vqa['labels'].values[idx]
            answer_text = vqa['Answer'].values[idx]

            sample['label'] = ans2idc
            sample['answer_text'] = answer_text
        else:
            sample['image_id'] = image_id
            
            
        return sample

    def __len__(self):

        return len(self.vqa)







In [55]:
def get_loader(input_dir, input_vqa_train, input_vqa_valid, img_feat_train, img_feat_valid, batch_size, num_workers,size=228):
    '''
        Load our dataset with dataloader for the train and valid data
    '''
    print('get_loader running')
    
    vqa_dataset = {
        'train': VqaDataset(
            input_dir=input_dir,
            input_vqa=input_vqa_train,
            img_feat_vqa=img_feat_train,
            phase = 'train'),
        'valid': VqaDataset(
            input_dir=input_dir,
            input_vqa=input_vqa_valid,
            img_feat_vqa=img_feat_valid,
            phase = 'valid')}
    

    data_loader = {
        phase: torch.utils.data.DataLoader(
            dataset=vqa_dataset[phase],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            )
        for phase in ['train','valid']}

    return data_loader

In [56]:
def get_test_loader(input_dir,input_test, img_feat_vqa,batch_size, num_workers,size=228):

   
    
    test_vqa_dataset = VqaDataset(
            input_dir=input_dir,
            input_vqa=input_test,
            img_feat_vqa=img_feat_vqa,
            phase = 'test')
    
    
    data_loader = torch.utils.data.DataLoader(dataset=test_vqa_dataset,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                num_workers=num_workers)
    return data_loader

In [57]:
def train_model():
    print('model training')
    since = time.time()

    best_acc1 = 0.0
    best_acc5 = 0.0
    
    #best_acc_val = 0.0
    best_epoch = 0
    list_train_loss_per_epoch = []
    list_valid_loss_per_epoch = []

    list_train_acc1_per_epoch = []
    list_valid_acc1_per_epoch = []

    model = VqaClassifierModel( opt=opt ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=opt.INIT_LERARNING_RATE, weight_decay=opt.LAMNDA) 

    input_dir = config.path_output_chd 
    input_vqa_train =config.input_vqa_train 
    input_vqa_valid =config.input_vqa_valid

    img_feat_train = config.img_feat_train
    img_feat_valid =config.img_feat_valid

    saved_dir = config.path_output_chd
  
    
    num_epochs = opt.NUM_EPOCHS
    image_size = opt.IMG_INPUT_SIZE
    num_workers = 0
    batch_size = opt.BATCH_SIZE


    # Create the DataLoader for our dataset
    # print('calling getloader')
    data_loader = get_loader(
        input_dir = input_dir , 
        input_vqa_train = input_vqa_train, 
        input_vqa_valid = input_vqa_valid,
        img_feat_train = img_feat_train, 
        img_feat_valid = img_feat_valid,
            batch_size = batch_size, 
            num_workers = num_workers,
            size = image_size )

    # print('calling end')
    # print('data_loader',data_loader)

    
    

    
    for epoch in range(opt.NUM_EPOCHS):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            #accuracy = 0
            top1_acc = 0
            top5_acc = 0
            acc_test_f = 0

            bleu = 0
            batch_step_size = len(data_loader[phase].dataset) / batch_size

            print('batch_step_size',batch_step_size)
        
            # Iterate over data.
            for batch_idx, batch_sample in enumerate(data_loader[phase]):
                
                # img = batch_sample['image'].to(device)
                image = batch_sample['image_feature'].to(device)
                questions = batch_sample['question']#.to(device)

                labels = batch_sample['label'].to(device)
                label_answer_text = batch_sample['answer_text']#.to(device)

                
                # print('image_shape',img.shape)
                print('image.shape',image.shape)
                print('questions.shape',type(questions))
                print('labels',type(labels))
                print('label_answer_text',type(label_answer_text))

                Q = torch.Tensor(labels)
                print('Q',Q.shape)
                # mixed_v, a_a, a_b, q_a, q_b, lam = mixup_data_vqa(
                #     batch_sample['image_feature'], Q, target, alpha=self.mixup, use_cuda=True
                # )
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    output= model(image, questions) 
                    print('type',type(output))
                    # print('output',output.data)
                  
                    _, preds = torch.max(output, 1)
                    
                   
                    loss = criterion(output, labels)
                   
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item()
                        
                #exact match score
                acc1, acc5 = accuracy(output.data, labels.data, topk=(1, 5))
                top1_acc += acc1
                top5_acc += acc5
                #bleu score
                b = get_bleu_score(preds, label_answer_text)
                bleu += b
                

                if batch_idx % 10 == 0:
                    print('| {} SET | Epoch [{:02d}/{:02d}], Step[{:04d}/{:04d}], Loss: {:.4f}, Top 1 Acc: {:.4f}, Top 5 Acc: {:.4f}, Bleu: {:.4f}'.format(phase.upper(), epoch+1, num_epochs, batch_idx, int(batch_step_size), loss.item(), acc1, acc5, b))#Acc: {:.4f},Bleu: {:.4f},acc, b


            epoch_loss = running_loss/batch_step_size
            epoch_acc1 = top1_acc/batch_step_size
            epoch_acc5 = top5_acc/batch_step_size
           
            epoch_blue = bleu/batch_step_size
            
            #save the loss and accuracy for train and valid
            if phase =='train':
              
                list_train_loss_per_epoch.append(epoch_loss)
                list_train_acc1_per_epoch.append(epoch_acc1)
               
            else:
                
                list_valid_loss_per_epoch.append(epoch_loss)
                list_valid_acc1_per_epoch.append(epoch_acc1)
                

            print('{} Loss: {:.4f} Top 1 Acc: {:.4f} Top 5 Acc: {:.4f} Bleu: {:.4f}'.format(
                phase, epoch_loss, epoch_acc1,epoch_acc5, epoch_blue))
            

            # deep copy the model
            if phase == 'valid' and epoch_acc1 > best_acc1: #or epoch_acc5 > best_acc5 ):
                best_acc1 = epoch_acc1
                best_acc5 = epoch_acc5
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())

               

    history_loss = {'train':list_train_loss_per_epoch, 'valid':list_valid_loss_per_epoch}
    history_acc1 = {'train':list_train_acc1_per_epoch, 'valid':list_valid_acc1_per_epoch}
  
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Top 1 Acc: {:4f}, Top 5 Acc: {:4f}'.format(best_acc1,best_acc5))

    #plot the loss and accuracy for train and valid
    # make_plot(history_loss, num_epochs, input_dir, type_plot='loss')
    # make_plot(history_acc1, num_epochs,input_dir, type_plot='acc1')
    
   

    # load best model weights
    model.load_state_dict(best_model_wts)
    model.load_state_dict(best_model_wts)
    state = {'epoch': best_epoch, 
            'state_dict': model.state_dict(), 
            'optimizer': optimizer.state_dict(), 
                'loss':epoch_loss,'valid_accuracy': best_acc1}
    
    full_model_path =saved_dir+'/model_state_seed_97.tar'
   
    torch.save(state, full_model_path)
    return model,history_loss,history_acc1,num_epochs,input_dir


In [58]:
model,history_loss,history_acc1,num_epochs,input_dir = train_model()

model training


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


get_loader running
Epoch 0/9
----------
batch_step_size 262.375
image.shape torch.Size([64, 1984, 1])
questions.shape <class 'list'>
labels <class 'torch.Tensor'>
label_answer_text <class 'list'>
Q torch.Size([64])
data 64
question_feature.shape torch.Size([64, 17, 768])
question_feature.shape torch.Size([64, 768, 17])
type <class 'torch.Tensor'>


../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [13,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [28,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [29,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
print(history_loss)

In [ ]:
history_loss['train']

In [ ]:
#plot the loss and accuracy for train and valid
make_plot(history_loss, num_epochs, input_dir, type_plot='loss')


In [ ]:
print(type(history_acc1))

In [ ]:
print(history_acc1)

In [ ]:
print(history_acc1['train'])

In [ ]:
train = history_acc1['train']

In [ ]:
l = []
for i in range(len(train)):
    l.append(train[i].cpu().numpy().tolist())

In [ ]:
l

In [ ]:
print(type(l[0]))

In [ ]:
l[0].shape

In [ ]:
print(type(train))

In [ ]:
train[0].to('cpu').numpy()

In [ ]:
print(type(history_acc1['train'][0]))

In [ ]:
history_acc1['train'][0].tolist()

In [ ]:
make_plot(history_acc1, num_epochs,input_dir, type_plot='acc1')